In [ ]:
!pip install networkx pyomo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.3 MB/s eta 0:00:00


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:20
🔁 Restarting kernel...


In [ ]:
!conda install glpk

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - \ done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.9.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - glpk


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.8.30  |       hbcca054_0         155 KB  conda-forge
    certifi-2024.8.30          |     pyhd8ed1ab_0         160 KB  conda-forge
    glpk-5.0                   |       h445213a_0        1023 KB  conda-forge
    gmp-6.3.0                  |       hac33072_2         449 KB  conda-forge
    libgcc-14.1.0              |       h77fa898_1         827 KB  conda-forge
    libgcc-ng-14.1.0           |       h69a702a_1          51 KB  conda-forge
    libgomp-14.1.0  

In [ ]:
import time
from itertools import cycle

import numpy as np
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx
import pyomo.environ as pyo
from pyomo.contrib.appsi.solvers.highs import Highs

In [ ]:
np.random.seed(42)  # Results should be always the same

N = 10
demands = np.random.randint(1, 10, size=N)
demands[0] = 0

capacity = 15
n_vehicles = 4

coordinates = np.random.rand(N, 2)
distances = squareform(pdist(coordinates, metric="euclidean"))
distances = np.round(distances, decimals=4)  # avoid numerical errors

Crear el modelo

In [ ]:
model = pyo.ConcreteModel("CVRP")

Crear el conjunto de índices para las sumatorias

In [ ]:
model.V = pyo.Set( initialize=range(len(demands)) )
model.A = pyo.Set( initialize=[(i, j) for i in model.V for j in model.V if i != j] )
model.K = pyo.Set( initialize=range(n_vehicles) )

Creación de parámetros del modelo

In [ ]:
model.Q = pyo.Param( initialize=capacity )
model.q = pyo.Param( model.V, initialize={i: d for (i, d) in enumerate(demands)} )
model.c = pyo.Param( model.A, initialize={(i, j): distances[i, j] for (i, j) in model.A} )

In [ ]:
model.x = pyo.Var( model.A, model.K, within = pyo.Binary )
model.y = pyo.Var( model.V, model.K, within = pyo.Binary )

In [ ]:
model.obj = pyo.Objective(
    expr=sum(
        model.x[i, j, k] * model.c[i, j]
        for (i, j) in model.A
        for k in model.K
    ),
    sense=pyo.minimize,
)

In [ ]:
for (i, j) in model.A:
  for k in model.K:
    print( model.x[i,j,k]*model.c[i, j] )

$$\sum_{k\in K} \sum_{ j \in V} x_{i,j,k} = \sum_{k\in K} \sum_{ j \in V} x_{j,i,k} = 1$$

In [ ]:
# ligaduras

def arcs_in(model, i):
    if i == model.V.first():
        return sum(model.x[:, i, :]) == len(model.K)
    else:
        return sum(model.x[:, i, :]) == 1.0


def arcs_out(model, i):
    if i == model.V.first():
        return sum(model.x[i, :, :]) == len(model.K)
    else:
        return sum(model.x[i, :, :]) == 1.0


def vehicle_assignment(model, i, k):
    return sum(model.x[:, i, k]) == model.y[i, k]


def comp_vehicle_assignment(model, i, k):
    return sum(model.x[i, :, k]) == model.y[i, k]


def capacity_constraint(model, k):
    return sum(model.y[i, k] * model.q[i] for i in model.V) <= model.Q

In [ ]:
model.arcs_in = pyo.Constraint(model.V, rule=arcs_in)
model.arcs_out = pyo.Constraint(model.V, rule=arcs_out)
model.vehicle_assignment = pyo.Constraint(model.V, model.K, rule=vehicle_assignment)
model.comp_vehicle_assignment = pyo.Constraint(model.V, model.K, rule=comp_vehicle_assignment)
model.capacity_constraint = pyo.Constraint(model.K, rule=capacity_constraint)

In [ ]:
def solve_step(model, solver):
    sol = solver.solve(model)
    return sol


def solve(model, solver):
    proceed = True
    while True:
        sol, proceed = solve_step(model, solver)
        print(sol)
    return sol

In [ ]:
solver = Highs()

solver.highs_options = {
    "log_file": "Highs.log",
    "mip_heuristic_effort": 0.2,
    "mip_detect_symmetry": True,
    "mip_rel_gap": 1e-6,
}

In [ ]:
pyo.SolverFactory('glpk').solve(model).write()

In [ ]:
model.display()

In [ ]:
!pip install mip

In [ ]:
from itertools import product
from sys import stdout as out
from mip import Model, xsum, minimize, BINARY

# names of places to visit
places = ['Antwerp', 'Bruges', 'C-Mine', 'Dinant', 'Ghent',
          'Grand-Place de Bruxelles', 'Hasselt', 'Leuven',
          'Mechelen', 'Mons', 'Montagne de Bueren', 'Namur',
          'Remouchamps', 'Waterloo']

# distances in an upper triangular matrix
dists = [[83, 81, 113, 52, 42, 73, 44, 23, 91, 105, 90, 124, 57],
         [161, 160, 39, 89, 151, 110, 90, 99, 177, 143, 193, 100],
         [90, 125, 82, 13, 57, 71, 123, 38, 72, 59, 82],
         [123, 77, 81, 71, 91, 72, 64, 24, 62, 63],
         [51, 114, 72, 54, 69, 139, 105, 155, 62],
         [70, 25, 22, 52, 90, 56, 105, 16],
         [45, 61, 111, 36, 61, 57, 70],
         [23, 71, 67, 48, 85, 29],
         [74, 89, 69, 107, 36],
         [117, 65, 125, 43],
         [54, 22, 84],
         [60, 44],
         [97],
         []]

# number of nodes and list of vertices
n, V = len(dists), set(range(len(dists)))

# distances matrix
c = [[0 if i == j
      else dists[i][j-i-1] if j > i
      else dists[j][i-j-1]
      for j in V] for i in V]

model = Model()

# binary variables indicating if arc (i,j) is used on the route or not
x = [ [model.add_var(var_type=BINARY) for j in V] for i in V ]

# continuous variable to prevent subtours: each city will have a
# different sequential id in the planned route except the first one
y = [ model.add_var() for i in V ]

# objective function: minimize the distance
model.objective = minimize(xsum(c[i][j]*x[i][j] for i in V for j in V))

# constraint : leave each city only once
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

# constraint : enter each city only once
for i in V:
    model += xsum(x[j][i] for j in V - {i}) == 1

# subtour elimination
for (i, j) in product(V - {0}, V - {0}):
    if i != j:
        model += y[i] - (n+1)*x[i][j] >= y[j]-n

# optimizing
model.optimize()

# checking if a solution was found
if model.num_solutions:
    out.write( 'route with total distance %g found: %s'% (model.objective_value, places[0]) )
    nc = 0
    while True:
        nc = [ i for i in V if x[nc][i].x >= 0.99 ][0]
        out.write(' -> %s' % places[nc])
        if nc == 0:
            break
    out.write('\n')

route with total distance 547 found: Antwerp -> Mechelen -> Leuven -> Hasselt -> C-Mine -> Montagne de Bueren -> Remouchamps -> Dinant -> Namur -> Mons -> Waterloo -> Grand-Place de Bruxelles -> Ghent -> Bruges -> Antwerp
